In [8]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry  # Correct import
import os



In [13]:
os.getcwd()
temp_df=pd.read_csv(r'Desktop/Zomato-Casestudy/data/combined_data.csv')

In [14]:
temp_df.head(50)

,NAME,PRICE,CUSINE_CATEGORY,CITY,REGION,URL,PAGE NO,CUSINE TYPE,TIMING,RATING_TYPE,RATING,VOTES
0,The Salt Cafe Kitchen & Bar,1000,"North Indian,Continental,Italian",Agra,Tajganj,https://www.zomato.com/agra/the-salt-cafe-kitc...,1,Casual Dining,1130am to 1130pm(Mon-Sun),Excellent,4.8,273
1,Bhagat Halwai,300,"Desserts,Bakery,Fast Food,South Indian",Agra,Civil Lines,https://www.zomato.com/agra/bhagat-halwai-civi...,1,Quick Bites,730am to 11pm(Mon-Sun),Very Good,4.3,554
2,Bikanervala,700,"North Indian,South Indian,Mithai,Street Food,D...",Agra,Khandari,https://www.zomato.com/agra/bikanervala-khanda...,1,Quick Bites,8am to 11pm(Mon-Sun),Very Good,4.4,645
3,Mama Chicken Mama Franky House,600,"North Indian,Mughlai,Rolls,Chinese,Fast Food,S...",Agra,Agra Cantt,https://www.zomato.com/agra/mama-chicken-mama-...,1,Quick Bites,"1230pm to 12midnight(Mon,Wed,Thu,Fri,Sat...",Very Good,4.3,894
4,Honeydew Restaurant,500,"North Indian,Fast Food,Chinese,South Indian",Agra,Dayal Bagh,https://www.zomato.com/agra/honeydew-restauran...,1,Quick Bites,"11am to 1035pm(Mon,Tue,Wed,Fri,Sat),11am to ...",Very Good,4.1,489
5,Cafe Turquoise Cottage,1600,"North Indian,Chinese,Continental,Italian,Pizza",Agra,Tajganj,https://www.zomato.com/agra/cafe-turquoise-cot...,1,Casual Dining,12noon to 1155pm(Mon-Sun),Excellent,4.7,294
6,Hichkee,1200,"Continental,North Indian,Chinese",Agra,Civil Lines,https://www.zomato.com/agra/hichkee-civil-line...,1,Casual Dining,4pm to 1am(Mon-Sun),Very Good,4.4,581
7,Domino's Pizza,700,"Pizza,Fast Food",Agra,Civil Lines,https://www.zomato.com/agra/dominos-pizza-civi...,1,Quick Bites,11am to 11pm(Mon-Sun),Good,3.9,583
8,Urban Deck,1000,"Mexican,Italian,North Indian",Agra,Khandari,https://www.zomato.com/agra/urban-deck-khandar...,1,Casual Dining,1pm to 12midnight(Mon-Sun),Very Good,4.3,608
9,Pinch Of Spice,1600,"North Indian,Chinese,Mughlai",Agra,Civil Lines,https://www.zomato.com/agra/pinch-of-spice-civ...,1,Casual Dining,"12noon to 4pm,7pm to 1130pm(Mon-Sun)",Excellent,4.5,863


In [19]:
import aiohttp
import asyncio


# Asynchronous function to fetch and process a single URL
async def fetch_url(session, url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        async with session.get(url, headers=headers) as response:
            if response.status == 202:
                text = await response.text()
                return extractlocations(text)
            else:
                logging.info(f"Non-202 status code received: {response.status} - URL: {url}")
                return np.nan
    except Exception as e:
        logging.error(f"Request failed: {e} - URL: {url}")
        return np.nan

# Function to extract location information from HTML
def extractlocations(html_text):
    match = re.search(r'center=([0-9\.\-]+,[0-9\.\-]+)', html_text)
    if match:
        return match.group(1)
    logging.warning("Image tag or location not found.")
    return np.nan

# Asynchronous function to handle all URLs in the DataFrame
async def fetch_all_urls(df):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in df['URL']:
            task = asyncio.ensure_future(fetch_url(session, url))
            tasks.append(task)
        results = await asyncio.gather(*tasks)
    return results

# Main function to enhance DataFrame with latitude and longitude
def enhanceDF(df):
    # Running asynchronous requests
    loop = asyncio.get_event_loop()
    locations = loop.run_until_complete(fetch_all_urls(df))

    # Splitting latitude and longitude from location strings
    latitudes, longitudes = [], []
    for loc in locations:
        if loc and isinstance(loc, str):
            try:
                lat, lon = map(float, loc.split(','))
                latitudes.append(lat)
                longitudes.append(lon)
            except ValueError:
                logging.error(f"Failed to convert location '{loc}' to float.")
                latitudes.append(np.nan)
                longitudes.append(np.nan)
        else:
            latitudes.append(np.nan)
            longitudes.append(np.nan)

    # Assigning the results back to the DataFrame
    df['latitude'] = latitudes
    df['longitude'] = longitudes

    # Dropping the URL column as required
    df = df.drop(columns=['URL'])

    return df

In [22]:
enhanceDF(temp_df)

RuntimeError: This event loop is already running